<a href="https://colab.research.google.com/github/Cosima10/master-thesis/blob/main/keyword_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## Preparation

---


#### <span style="color:pink"> USE THIS MODULE IF YOU ARE RUNNING IT IN GOOGLE COLAB </span>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import time
!pip install selenium
from selenium import webdriver
import sys
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
! pip install selenium, webdriver_manager
! apt-get update
! apt install chromium-chromedriver
! cp / usr/lib/chromium-browser/chromedriver / usr/bin
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver = wd
import os

path_source = "/content/drive/MyDrive/scraper/df link.xlsx"
path_output = "/content/drive/MyDrive/scraper/result/"
if not os.path.exists(path_output):
    os.makedirs(path_output)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 4.9 MB/s 
     |████████████████████████████████| 359 kB 50.6 MB/s 
     |████████████████████████████████| 138 kB 49.6 MB/s 
     |████████████████████████████████| 4.0 MB 40.7 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: Invalid 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


#### <span style="color:pink"> USE THIS MODULE IF YOU ARE RUNNING IT ON YOUR LOCAL MACHINE </span>

In [8]:
# !pip install selenium, webdriver_manager, numpy, pandas
# import time
# import numpy as np
# from bs4 import BeautifulSoup
# import pandas as pd
# import sys
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# import os
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# wd = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
# driver = wd
# cwd = os.getcwd()
# cwd

# path_source = cwd+"/df link.xlsx"
# path_output = cwd+"/result/"
# if not os.path.exists(path_output):
#     os.makedirs(path_output)


---

# SCRAPER

---

In [3]:
# initialising string

def hapusspecialcharacter(ini_string):
    getVals = list([val for val in ini_string
                    if val.isalpha() or val.isnumeric() or val == " "])
    result = "".join(getVals)
    return result


def unique(list1):
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return(unique_list)


def years_check(year, df):
    number_of_news_in_year = len(df[df['date'].str.contains(str(year))])
    return number_of_news_in_year

df_link = pd.read_excel(path_source)

In [4]:
# create output folders and files 

def createExcelFiles(iprocess, kamus, keyword):
    if not os.path.exists(f"{path_output}/{df_link['country'][iprocess]}"):
        os.makedirs(f"{path_output}/{df_link['country'][iprocess]}")

    results_path = f"{path_output}/{df_link['country'][iprocess]}/"
    
    df = pd.DataFrame(kamus).astype(str)
    file_name = f"{results_path}Links {df_link['country'][iprocess]} {df_link['number'][iprocess]} {df_link['link'][iprocess].split('/')[2].replace('.','_')}.xlsx"
    # if file exist
    if os.path.exists(file_name):   mode = 'a'
    else: mode = 'w'
    with pd.ExcelWriter(file_name, mode=mode, engine="openpyxl") as writer:
        df.to_excel(writer, sheet_name=keyword, index=False)

    yearList = []
    counterList = []
    for i in range(2023)[-6:]:
        yearList.append(i)
        counterList.append(years_check(i, df))

    file_nameCek = f"{results_path}Stats {df_link['country'][iprocess]} {df_link['number'][iprocess]} {df_link['link'][iprocess].split('/')[2].replace('.','_')}.xlsx"
    kamusCek = {"year": yearList, "Number of Related News": counterList}
    dfCek = pd.DataFrame(kamusCek).astype(str)
    if os.path.exists(file_nameCek):   mode = 'a'
    else: mode = 'w'
    with pd.ExcelWriter(file_nameCek, mode=mode, engine="openpyxl") as writer:
        dfCek.to_excel(writer, sheet_name=keyword, index=False)
    return dfCek

---

## DECORATOR
replacing the common part of each scraper and therewith reducing redundancy

---

In [5]:
def scrap_data(params):
    def scrap_decorator(fn):
        tick = time.time()
        iprocess = params.get('iprocess')
        url = params.get('url')
        pagination = params.get('pagination')

        def wrapper(*args, **kwargs):
            keyword = args[0] if args else kwargs.get('keyword', None)
            news_links = []
            dates = []
            for ipage in pagination:
                wd.get(url.format(keyword=keyword, ipage=ipage))  # link with search query
                content = driver.page_source.encode('utf-8').strip()
                soup = BeautifulSoup(content)
                news_links, dates, found = fn(soup, ipage, news_links, dates)
                if not found:
                    print(f"[PAGE NO : {ipage}] [NO MORE ARTICLES FOUND]")
                    break
                
            kamus = {"news link": news_links, "date": dates}
            print(f'[Website: {url.split("/")[2]}] [Keyword: {keyword}] [Time Taken: {round(time.time() - tick, 2)} sec]')
            return createExcelFiles(iprocess, kamus, keyword)

        return wrapper
    return scrap_decorator

---

# GERMANY

---

#### 1) NTV

In [7]:
@scrap_data(params={'iprocess': 7, 'url': 'https://www.ntv.de/suche/?q={keyword}&at=all&page={ipage}', 'pagination': list(range(1,21))})
def germany1ntv(soup, ipage, news_links, dates):
    found = True
    b = soup.find('div', class_='search__results').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", "teaser__date").text.split()))
    return news_links, dates, found

In [15]:
#TEST 
#print(germany1ntv("dsgvo"))

[PAGE NO : 1] [ARTICLES FOUND: 20]
[PAGE NO : 2] [ARTICLES FOUND: 20]
[PAGE NO : 3] [ARTICLES FOUND: 20]
[PAGE NO : 4] [ARTICLES FOUND: 20]
[PAGE NO : 5] [ARTICLES FOUND: 20]
[PAGE NO : 6] [ARTICLES FOUND: 20]
[PAGE NO : 7] [ARTICLES FOUND: 20]
[PAGE NO : 8] [ARTICLES FOUND: 20]
[PAGE NO : 9] [ARTICLES FOUND: 14]
[PAGE NO : 10] [NO MORE ARTICLES FOUND]
   year Number of Related News
0  2017                      3
1  2018                     91
2  2019                     34
3  2020                     15
4  2021                     23
5  2022                      7


#### 2) Deutsche Welle

In [11]:
@scrap_data(params={'iprocess': 9, 'url': 'https://www.dw.com/search/?languageCode=en&item={keyword}&searchNavigationId=9097&sort=DATE&resultsCounter=10000', 'pagination': list(range(1,21))})
def germany2dw(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "searchResult")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", "date").text.split()))
    return news_links, dates, found

#### 3) Frankfurter Allgemeine Zeitung

In [12]:
@scrap_data(params={'iprocess': 10, 'url': 'https://www.faz.net/suche/s{ipage}.html?ct=article&ct=audio&ct=blog&ct=gallery&ct=infografik&ct=storytelling&ct=video&&query={keyword}#listPagination', 'pagination': list(range(1,21))})
def germany3faz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("li", "lst-Teaser_Item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

---

# DENMARK

---

#### 1) INFORMATION.DK

In [13]:
@scrap_data(params={'iprocess': 0, 'url': 'https://www.information.dk/search/site/{keyword}?page={ipage}', 'pagination': list(range(1,21))})
def denmark1information(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", {"typeof": "sioc:Item foaf:Document"})
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.information.dk"+i.find("a").get("href"))
            dates.append(" ".join(i.find("li", "date first").text.split()))
    return news_links, dates, found

#### 2) POLITIKEN.DK

In [14]:
@scrap_data(params={'iprocess': 1, 'url': 'https://politiken.dk/search/?q={keyword}&target=pol&sort=pd&page={ipage}', 'pagination': list(range(1,21))})
def denmark2politiken(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "search-result__article u-padding--vertical-normal")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://politiken.dk/"+i.find("a").get("href"))
            dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

#### 3) BERLINGSKE.DK

In [15]:
@scrap_data(params={'iprocess': 4, 'url': 'https://www.berlingske.dk/search?query={keyword}&limit=10&offset={ipage}0', 'pagination': list(range(1,21))})
def denmark3berlingske(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "teaser__description align-self-top w-100")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.berlingske.dk/"+i.find("a").get("href"))
            dates.append(" ".join(i.find("div","teaser__date").text.split()))
    return news_links, dates, found

#### 4) JV.DK

In [ ]:
@scrap_data(params={'iprocess': 2, 'url': 'https://jv.dk/soeg?keywords={keyword}&limit=10&offset={ipage}0', 'pagination': list(range(1,21))})
def denmark4jv(soup, ipage, news_links, dates):
    found = True
    b = soup.find('div', class_='page-block').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://jv.dk/"+i.find("a").get("href"))
            dates.append(" ".join(i.find("time", "search-result__date").text.split()))
    return news_links, dates, found

---

# AUSTRIA

---

#### 1) WIENER ZEITUNG

In [16]:
@scrap_data(params={'iprocess': 23, 'url': 'https://www.wienerzeitung.at/suche/?such={keyword}&date_min=20220112&date_max=20220412&em_page={ipage}', 'pagination': list(range(1,21))})
def austria1wienerzeitung(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "col-12 col-md-4 em_mobile_list_layout")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

#### 2) KLEINE ZEITUNG

In [17]:
@scrap_data(params={'iprocess': 24, 'url': 'https://www.kleinezeitung.at/user/search.do?action=1&resultsPage={ipage}&resetForm=0&type=4196%2C4193%2C4206&searchText={keyword}', 'pagination': list(range(1,21))})
def austria2kleinezeitung(soup, ipage, news_links, dates):
    found = True
    b = soup.find('section', class_='article-card--search').find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
        try:
            dates.append(" ".join(i.find("small", "article-card__date").text.split()))
        except: dates.append(" ".join(""))
    return news_links, dates, found

#### 3) DIE PRESSE

In [18]:
@scrap_data(params={'iprocess': 26, 'url': 'https://www.diepresse.com/suche?s={keyword}&p={ipage}', 'pagination': list(range(1,21))})
def austria3diepresse(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", "card__content")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            try:    dates.append(" ".join(i.find("div", "card__header").get_text().split()))
            except: dates.append("")
    return news_links, dates, found

---

# PORTUGAL

---

#### 1) OBSERVADOR

In [6]:
def portugal1observador(keyword):
    news_links = []
    dates = []
    iprocess = 31
    print("[WEBSITE: OBSERVADOR]")
    wd.get(f"https://observador.pt/pesquisa/?q={keyword}")    
    for ipage in list(range(21))[1:]:
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content)
        b = soup.find_all("div", "gsc-webResult gsc-result")
        print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
        for i in b:
            link = i.find("a")
            if link:
                news_links.append(""+i.find("a").get("href"))
                try:
                    dates.append(" ".join(i.find("div", "gs-visibleUrl-breadcrumb").find_all('span')[1].text.split()))
                except:
                    dates.append("")
                try:
                    wd.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[ipage]'.replace("ipage", str(ipage))).click()
                    time.sleep(2)
                except:
                    y = 0
    kamus = {"news link": news_links, "date": dates}
    return createExcelFiles(iprocess, kamus, keyword)


#### 2) DN.PT

In [7]:
def portugal2dn(keyword):
    news_links = []
    dates = []
    iprocess = 30
    print("[WEBSITE: DN]")
    wd.get(f"https://www.dn.pt/pesquisa.html?q={keyword}")    
    for ipage in list(range(21))[1:]:
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content)
        b = soup.find_all("div", "gsc-webResult gsc-result")
        print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
        for i in b:
            link = i.find("a")
            if link:
                news_links.append(""+i.find("a").get("href"))
                try:
                    dates.append(" ".join(i.find("div", "gs-bidi-start-align gs-snippet").text.split('...')[0].split()))
                except:
                    dates.append("")
                try:
                    time.sleep(1)
                    wd.find_element_by_xpath('//*[@id="___gcse_0"]/div/div/div/div[5]/div[2]/div/div/div[2]/div/div[ipage]'.replace("ipage", str(ipage))).click()
                    time.sleep(2)
                except:
                    y = 0
    kamus = {"news link": news_links, "date": dates}
    return createExcelFiles(iprocess, kamus, keyword)

#### 3) DNOTICIAS.PT

In [8]:
@scrap_data(params={'iprocess': 32, 'url': 'https://pesquisa.dnoticias.pt/elastic/results/?search={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def portugal3dnoticias(soup, ipage, news_links, dates):
    found = True
    b = soup.find('ul', class_='search-list-results').find_all('li')
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", "search-result-date").text.split()))
    return news_links, dates, found

---

# ITALY

---

#### 1) REPUBBLICA.IT

In [6]:
@scrap_data(params={'iprocess': 33, 'url': 'https://ricerca.repubblica.it/ricerca/repubblica?query={keyword}&view=repubblica&ref=HRHS&page={ipage}', 'pagination': list(range(1,21))})
def italy1repubblica(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

#### 2) TODAY.IT

In [6]:
@scrap_data(params={'iprocess': 40, 'url': 'https://www.today.it/search/query/{keyword}/pag/{ipage}', 'pagination': list(range(1,21))})
def italy2today(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("span", class_='c-story__byline').find('a').get("href"))
            dates.append(" ".join(i.find("span", class_='c-story__byline').text.split(',')[-1].split()))
    return news_links, dates, found

#### 3) ILPOST.IT

In [10]:
@scrap_data(params={'iprocess': 39, 'url': 'https://www.ilpost.it/search_gcse/?q={keyword}#gsc.tab=0&gsc.q={keyword}&gsc.page={ipage}', 'pagination': list(range(1,21))})
def italy3ilpost(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_='gsc-webResult gsc-result')
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find('a').get("href"))
            dates.append(" ".join(i.find("div", class_='gs-visibleUrl-breadcrumb').find_all('span')[1].text.split()))
    return news_links, dates, found

---

# BULGARIA

---

#### 1) VESTI.BG

In [25]:
@scrap_data(params={'iprocess': 65, 'url': 'https://www.vesti.bg/tarsene?q={keyword}', 'pagination': list(range(1,2))})
def bulgaria1vesiti(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="list-item list-item-category")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find('a').get("href"))
            dates.append(" ".join(i.find("span", class_="label-time").text.split()))
    return news_links, dates, found

#### 2) 24CHASA.BG

In [26]:
@scrap_data(params={'iprocess': 66, 'url': 'https://www.24chasa.bg/Search?what={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria2chasa(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="entry-short")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find('a').get("href"))
            dates.append(" ".join(i.find("span", class_="article-date").text.split('|')[-1].split()))
    return news_links, dates, found

#### 3) BLITZ.BG

In [27]:
@scrap_data(params={'iprocess': 67, 'url': 'https://blitz.bg/search?q={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria3blitz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all("div", class_="tech-news-content mt-0")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find('a').get("href"))
            dates.append(" ".join(i.find_all("p")[-1].find("span").text.split()))
    return news_links, dates, found

#### 4) FAKTI.BG

In [ ]:
@scrap_data(params={'iprocess': 68, 'url': 'https://fakti.bg/search?q={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def bulgaria4fakti(soup, ipage, news_links, dates):
    found = True
    b = soup.find('section', id='main').find_all("li")[:21]
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://fakti.bg"+i.find("a").get("href"))
            dates.append(" ".join(i.find("div", class_="ndtvc").find("div").text.split()))
    return news_links, dates, found

---
# CROATIA
---


#### 1) INDEX.HR

In [28]:
@scrap_data(params={'iprocess': 58, 'url': 'https://www.index.hr/trazi.aspx?upit={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def croatia1index(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="grid-items-list")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.index.hr"+i.find("a").get("href"))
            dates.append(" ".join(i.find("div", class_="publish-date").text.split()))
    return news_links, dates, found

#### 2) TPORTAL.HR

In [29]:
@scrap_data(params={'iprocess': 62, 'url': 'https://www.tportal.hr/pretrazivanje?query={keyword}&page={ipage}', 'pagination': list(range(1,21))})
def croatia2tportal(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="articlePreviewListType10")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.tportal.hr"+i.find("a").get("href"))
            dates.append(" ".join(i.find("h2", class_="title publishedSort").text.split()))
    return news_links, dates, found

#### 3) DNEVNO.HR

In [30]:
@scrap_data(params={'iprocess': 64, 'url': 'https://www.dnevno.hr/?s={keyword}', 'pagination': list(range(1,2))})
def croatia3dnevno(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('article', class_="post")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("time", class_="date").text.split()))
    return news_links, dates, found

---
# Greece
---

#### 1) PROTOTHEMA.GR

In [31]:
@scrap_data(params={'iprocess': 49, 'url': 'https://www.protothema.gr/ajax/Atcom.Sites.ProtoThema.Components.Search.Page/?pg={ipage}&q={keyword}', 'pagination': list(range(1,21))})
def greece1protothema(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", class_="update_well").text.split(',')[0].split()))
    return news_links, dates, found

#### 2) IN.GR

In [32]:
@scrap_data(params={'iprocess': 51, 'url': 'https://www.in.gr/search-results/3/?q={keyword}#gsc.tab=0&gsc.q={keyword}&gsc.page={ipage}', 'pagination': list(range(1,21))})
def greece2in(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="gsc-webResult gsc-result")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("div", class_="gs-bidi-start-align gs-snippet").text.split('...')[0].split()))
    return news_links, dates, found

#### 3) NEWSIT.GR

In [33]:
@scrap_data(params={'iprocess': 53, 'url': 'https://www.newsit.gr/page/{ipage}/?s={keyword}', 'pagination': list(range(1,21))})
def greece3newsit(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('article', class_="blog-article")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("time", class_="entry-date published").text.split()))
    return news_links, dates, found

#### 4) DIKAILOGIKA.GR

In [ ]:
@scrap_data(params={'iprocess': 52, 'url': 'https://www.dikaiologitika.gr/site/search/{keyword}?searchword={keyword}&start={ipage}', 'pagination': list(map(lambda x: x*20,range(21)))})
def greece4dikaiologitika(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="itemContainer")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.dikaiologitika.gr"+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", class_="genericItemDateCreated").text.split('-')[0].split()))
    return news_links, dates, found

---
# SLOVAKIA
---

#### 1) TOPKY.SK

In [34]:
@scrap_data(params={'iprocess': 44, 'url': 'https://www.topky.sk/search/{keyword}/{ipage}/', 'pagination': list(range(1,21))})
def slovakia1topky(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="entry")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.topky.sk"+i.find("a").get("href"))
            dates.append(" ".join(i.find("p", class_="time").text.split(' ')[0].split()))
    return news_links, dates, found

#### 2) TERAZ.SK

In [35]:
@scrap_data(params={'iprocess': 47, 'url': 'https://www.teraz.sk/search?q={keyword}&p={ipage}', 'pagination': list(range(1,21))})
def slovakia2teraz(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('div', class_="mediaListing-item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append("https://www.teraz.sk"+i.find("a").get("href"))
            dates.append(" ".join(i.find("time").text.split()))
    return news_links, dates, found

#### 3) AKTUALITY.SK

In [36]:
@scrap_data(params={'iprocess': 42, 'url': 'https://www.aktuality.sk/vyhladavanie/{ipage}/?search%5Btext%5D={keyword}&search%5Bzdroj%5D=spravy', 'pagination': list(range(1,21))})
def slovakia3aktuality(soup, ipage, news_links, dates):
    found = True
    b = soup.find_all('li', class_="article-item list-item")
    if len(b) == 0: found = False
    else: print(f"[PAGE NO : {ipage}] [ARTICLES FOUND: {len(b)}]")
    for i in b:
        link = i.find("a")
        if link:
            news_links.append(""+i.find("a").get("href"))
            dates.append(" ".join(i.find("span", class_="article-time").text.split(' ')[0].split()))
    return news_links, dates, found

---

# OUTPUT 

---

Each time, one will run the function for a certain news site of a country, it will output two excel files with the following data:

- Excel file 1: number of news articles found for each keyword per year (2017-2022), each keyword is in a different sheet within the same excel file,
- Excel file 2: links of news articles per date (for cross-checks) separated for each keyword in a different sheet.

Naming conventions for the files that you will find in the folder 'results' > country_name:

- Result + country_name + news_site_name: statistics 

- country_name + news_site_name: links for cross-checking


---

# RUN THE CODE

---

In [ ]:
DK_keyword_list = ["gdpr", "privatliv", "Databeskyttelsesloven", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in DK_keyword_list:
    print(denmark1information(keyword))
    print(denmark2politiken(keyword))
    print(denmark3berlingske(keyword))
    #print(denmark4jv(keyword))

In [13]:
GER_keyword_list = ["gdpr", "dsgvo", "datenschutz", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in GER_keyword_list:
    print(germany1ntv(keyword))
    print(germany2dw(keyword))
    print(germany3faz(keyword))

In [ ]:
AT_keyword_list = ["gdpr", "dsgvo", "datenschutz", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in AT_keyword_list:
    print(austria1wienerzeitung(keyword))
    print(austria2kleinezeitung(keyword))
    print(austria3diepresse(keyword))

In [ ]:
PT_keyword_list = ["gdpr", "Lei n.º 58/2019", "privacidade", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in PT_keyword_list:
    print(portugal1observador(keyword))
    print(portugal2dn(keyword))
    print(portugal3dnoticias(keyword))

In [ ]:
IT_keyword_list = ["gdpr", "Codice in materia di protezione dei dati personali", "protezione dei dati", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in IT_keyword_list:
    print(italy1repubblica(keyword))
    #print(italy2today(keyword)) # EXCEPTION: apparently only this news site has big trouble in searching for German terms, that's why please run it afterwards with its separate keyword_list2
    print(italy3ilpost(keyword))

In [ ]:
SL_keyword_list = ["gdpr", "Zàkon o ochrane osobných údajov", "súkromia", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in SL_keyword_list:
    print(slovakia1topky(keyword))
    print(slovakia2teraz(keyword))
    print(slovakia3aktuality(keyword)) 

In [ ]:
BL_keyword_list = ["gdpr", "Закон за защита на личните данни", "поверителност", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in BL_keyword_list:
    print(bulgaria1vesiti(keyword))
    print(bulgaria2chasa(keyword))
    print(bulgaria3blitz(keyword))
    #print(bulgaria4fakti(keyword))

In [ ]:
GR_keyword_list = ["gdpr", "μυστικότητα", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in GR_keyword_list:
    print(greece1protothema(keyword))
    print(greece2in(keyword))
    print(greece3newsit(keyword))
    #print(greece4dikaiologitika(keyword))

In [ ]:
HR_keyword_list = ["gdpr", "Zakon o Provedbi Opće Uredbe o Zaštiti Podataka", "privatnost", "web tracking", "online tracking", "digital analytics", "data protection authorities"]

for keyword in HR_keyword_list:
    print(croatia1index(keyword))
    print(croatia2tportal(keyword))
    print(croatia3dnevno(keyword))

In [9]:
IT_keyword_list = ["gdpr","web tracking","online tracking", "digital analytics", "data protection authorities"]

for keyword in IT_keyword_list:
      print(italy1repubblica(keyword))
      print(italy2today(keyword)
      print(italy3ilpost(keyword))

[PAGE NO : 1] [ARTICLES FOUND: 15]
[PAGE NO : 2] [ARTICLES FOUND: 15]
[PAGE NO : 3] [ARTICLES FOUND: 15]
[PAGE NO : 4] [ARTICLES FOUND: 15]
[PAGE NO : 5] [ARTICLES FOUND: 15]
[PAGE NO : 6] [ARTICLES FOUND: 15]
[PAGE NO : 7] [ARTICLES FOUND: 15]
[PAGE NO : 8] [ARTICLES FOUND: 15]
[PAGE NO : 9] [NO MORE ARTICLES FOUND]
[Website: www.today.it] [Keyword: gdpr] [Time Taken: 131.59 sec]
   year Number of Related News
0  2017                      0
1  2018                     26
2  2019                     29
3  2020                     25
4  2021                     25
5  2022                     15
[PAGE NO : 1] [ARTICLES FOUND: 12]
[PAGE NO : 2] [NO MORE ARTICLES FOUND]
[Website: www.today.it] [Keyword: web tracking] [Time Taken: 133.95 sec]
   year Number of Related News
0  2017                      1
1  2018                      1
2  2019                      2
3  2020                      2
4  2021                      3
5  2022                      3
[PAGE NO : 1] [ARTICLES FOUND: 15]
